In [3]:
# 개체명 인식(NER, Named Entity Recognition)
# - BIO(Beginning, Inside, Outside) 
#     B-개체명 : 개체명이 시작되는 단어
#     I-개체명 : B-개체명과 연결되는 단어
#     O-개체명 : 개체명 이외의 모든 것


# To Do! 양방향 LSTM을 이용한 개체명 인식기(NER)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

# train.txt 데이터 설명
# ; => 원본 문장
# $ => NER 처리 결과
# 총 4열 => 토큰 번호/단어 토큰/품사 태그/BIO 태그

def read_file(file_name):
  sents = []
  with open(file_name, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
    for idx, l in enumerate(lines):
      if l[0] == ';' and lines[idx + 1][0] == '$':
        this_sent = []
      elif l[0] == '$' and lines[idx - 1][0] == ';':
        continue
      elif l[0] == '\n':
        sents.append(this_sent)
      else:
        this_sent.append(tuple(l.split()))

  return sents

corpus = read_file('./data/train.txt')

# 단어, BIO 태그 열만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
  tagged_sentence = []
  sentence, bio_tag = [], []
  for w in t:
    tagged_sentence.append((w[1], w[3]))
    sentence.append(w[1])
    bio_tag.append(w[3])

  sentences.append(sentence)
  tags.append(bio_tag)

print('샘플 크기 :', len(sentences))

샘플 크기 : 3555


In [4]:
print(sentences[0]) # 단어

['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']


In [5]:
print(tags[0]) # BIO 태그

['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [6]:
print('샘플 문장 시퀀스 최대 길이 :', max(len(l) for l in sentences))
print('샘플 문장 시퀀스 평균 길이 :', (sum(map(len, sentences))/len(sentences))) # -> 시퀀스 패딩 크기 결정

샘플 문장 시퀀스 최대 길이 : 168
샘플 문장 시퀀스 평균 길이 : 34.03909985935302


In [7]:
# 토크나이저 정의
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='00V') # out of vocabulary : 단어 사전에 포함되지 않은 단어
sent_tokenizer.fit_on_texts(sentences)

tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 소문자 X
tag_tokenizer.fit_on_texts(tags)


vocab_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1

print('단어 사전 크기 :', vocab_size)
print('BIO 태그 사전 크기 :', tag_size)

단어 사전 크기 : 13834
BIO 태그 사전 크기 : 8


In [8]:
# 학습용 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)

print(x_train[0])
print(y_train[0])

[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3]
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [9]:
index_to_word = sent_tokenizer.index_word # 시퀀스 인덱스 -> 단어 변환
index_to_ner = tag_tokenizer.index_word   # 시퀀스 인덱스 -> NER 변환

print(index_to_word[2])
print(index_to_ner)

하
{1: 'O', 2: 'I', 3: 'B_OG', 4: 'B_PS', 5: 'B_DT', 6: 'B_LC', 7: 'B_TI'}


In [10]:
index_to_ner[0] = 'PAD'
print(index_to_ner)

{1: 'O', 2: 'I', 3: 'B_OG', 4: 'B_PS', 5: 'B_DT', 6: 'B_LC', 7: 'B_TI', 0: 'PAD'}


In [11]:
# 시퀀스 패딩 처리 : 입출력 벡터 크기를 동일하게 맞추기 위해

max_len = 40 # 단어 시퀀스 평균 길이보다 넉넉하게 정의
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

print(x_train[0])
print(y_train[0])

[ 183   11 4276  884  162  931  402   10 2608    7 1516  608  145 1361
  414    4 6347    2    8    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[1 1 1 1 1 3 2 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [12]:
# train:test = 8:2
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, 
                                                    test_size=.2, random_state=0)

# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print('x_train.shape :', x_train.shape)
print('y_train.shape :', y_train.shape)
print('x_test.shape :', x_test.shape)
print('y_test.shape :', y_test.shape)

x_train.shape : (2844, 40)
y_train.shape : (2844, 40, 8)
x_test.shape : (711, 40)
y_test.shape : (711, 40, 8)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential() # 순차 모델
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax'))) # tag_size의 출력 뉴런에서 제일 확률 높은 출력값 1개를 선택하기 때문에 활성화 함수로 softmax 사용.
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
23/23 [==============================] - 28s 889ms/step - loss: 0.5160 - accuracy: 0.8362
Epoch 2/10
23/23 [==============================] - 21s 894ms/step - loss: 0.2344 - accuracy: 0.8957
Epoch 3/10
23/23 [==============================] - 20s 888ms/step - loss: 0.1581 - accuracy: 0.9237
Epoch 4/10
23/23 [==============================] - 20s 887ms/step - loss: 0.1159 - accuracy: 0.9439
Epoch 5/10
23/23 [==============================] - 20s 884ms/step - loss: 0.0825 - accuracy: 0.9625
Epoch 6/10
23/23 [==============================] - 21s 891ms/step - loss: 0.0580 - accuracy: 0.9747
Epoch 7/10
23/23 [==============================] - 21s 894ms/step - loss: 0.0416 - accuracy: 0.9815
Epoch 8/10
23/23 [==============================] - 20s 889ms/step - loss: 0.0331 - accuracy: 0.9848
Epoch 9/10
23/23 [==============================] - 20s 881ms/step - loss: 0.0269 - accuracy: 0.9878
Epoch 10/10
23/23 [==============================] - 21s 891ms/step - loss: 0.0225 - accura

In [14]:
print('평과 결과 :', model.evaluate(x_test, y_test)[1])

23/23 [==============================] - 2s 55ms/step - loss: 0.2269 - accuracy: 0.9399
평과 결과 : 0.9399349689483643


In [15]:
def sequences_to_tag(sequences):
  result = []
  for sequence in sequences:
    temp = []
    for pred in sequence:
      pred_index = np.argmax(pred)
      temp.append(index_to_ner[pred_index].replace('PAD', 'O'))
    result.append(temp)

  return result

y_predicted = model.predict(x_test) 

# 예측 NER
pred_tags = sequences_to_tag(y_predicted)
print(pred_tags[0])

['B_DT', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [16]:
# 실제 NER
test_tags = sequences_to_tag(y_test)
print(test_tags[0])

['B_DT', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [19]:
# pip install seqeval
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags)) # 정밀도(precision), 재현율(recall), F1-score 출력 

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

           _       0.70      0.51      0.59       657
         _DT       0.92      0.90      0.91       335
         _LC       0.75      0.55      0.64       312
         _OG       0.81      0.50      0.62       481
         _PS       0.76      0.44      0.55       374
         _TI       0.88      0.79      0.83        66

   micro avg       0.79      0.57      0.66      2225
   macro avg       0.80      0.62      0.69      2225
weighted avg       0.78      0.57      0.65      2225



In [21]:
print('F1-score: {:.1%}'.format(f1_score(test_tags, pred_tags)) # 예측 결과의 평균 F1-score

F1-score: 66.1%


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [22]:
# 학습이 완료된 모델에 학습되지 않은 새로운 유형의 문장의 개체명 예측
word_to_index = sent_tokenizer.word_index
new_sentence = '삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.'.split()
new_x = []
for w in new_sentence:
  try:
    new_x.append(word_to_index.get(w, 1))
  except KeyError:
    new_x.append(word_to_index['OOV'])

print('새로운 유형의 시퀀스 :', new_x)

새로운 유형의 시퀀스 : [531, 307, 1476, 286, 1507, 6766, 1]


In [23]:
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding='post',
                                                       value=0, maxlen=max_len)
p = model.predict(np.array([new_padded_seqs[0]]))                                                      
p = np.argmax(p, axis=-1)

print('{:10} {:5}'.format('단어', '예측 NER'))
print('-' * 50)
for w, pred in zip(new_sentence, p[0]):
  print('{:10} {:5}'.format(w, index_to_ner[pred]))

단어         예측 NER
--------------------------------------------------
삼성전자       B_OG 
출시         O    
스마트폰       O    
오늘         B_DT 
애플         B_OG 
도전장        I    
내밀다.       I    
